# Calculation of AUROCs from Single-sample GSEA projection (ssGSEA)

## Background

Traditional gene set enrichment analysis assesses the differential coordinate up- or down-regulation of a biological process or pathway between groups of samples belonging to two phenotypes. The ability to assess that enrichment in individual samples, especially independently of pre-assigned phenotype labels, provides the opportunity to analyze transcription data at a higher level, by using gene sets/pathways instead of genes, resulting in a much more biologically interpretable set of features. Single-sample Gene Set Enrichment Analysis (ssGSEA) Projection accomplishes this.

**ssGSEA projects a single sample’s full gene expression profile into the space of gene sets**. It does this via *enrichment scores*, which represent the degree to which the genes in each particular gene set are coordinately up- or down- regulated within that sample.  

Any supervised or unsupervised machine learning technique or other statistical analysis can then be applied to the resulting projected dataset. The benefit is that the **ssGSEA projection transforms the data to a higher-level (pathways instead of genes) space representing a more biologically interpretable set of features on which analytic methods can be applied.**

In this notebook we will be using the ssGSEA scores and a binary phenotype to calculate the receiver operating characteristic (ROC) curves for each gene set, and the area under the ROC curve. This metric is useful for determining if the ennrichment score for a given gene set is capable of serving as a classification metric for the selected phenotype.
Since ssGSEA scores are determined on a per-sample basis, and each sample is inndividually normalized (for example, by transcripts per million), once a gene set has been determined to be a good classifier, new samples can be classified based on their enrichment score if their true phenotype is unknown.

## Before you begin

You must log in to a GenePattern server. In this notebook we will use **```GenePattern Cloud``` **

Note: if you are not familiar with GenePattern Notebook features, you can revise them here: <a href="https://notebook.genepattern.org/services/sharing/notebooks/361/preview/">GenePattern Notebook Tutorial</a>.

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
Sign in to GenePattern by entering your username and password into the form below.
</div>

In [28]:
# Requires GenePattern Notebook: pip install genepattern-notebook
import gp
import genepattern

# Username and password removed for security reasons.
genepattern.display(genepattern.session.register("https://beta.genepattern.org/gp", "", ""))

GPAuthWidget()

## Project gene expression dataset into the space of selected gene sets

We will use the GenePattern ssGSEA analysis to transform a gene expression dataset into a dataset where each row corresponds to a pathway, and each column is a sample. Each value in the new dataset will therefore represent the up- or downregulation of a pathway (row) within a sample (column).

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>

Provide the required parameters for the ssGSEA module below.

- For the **input gct file*** parameter, Provide a file in the [GCT format](https://genepattern.org/file-formats-guide#GCT')  
     - For example: <a href="https://datasets.genepattern.org/data/ccmi_tutorial/2017-12-15/BRCA_HUGO_symbols.preprocessed.gct" target="_blank">BRCA_HUGO_symbols.preprocessed.gct</a>
- For a detailed description of the parameters you can read the <a href='https://gsea-msigdb.github.io/ssGSEA-gpmodule/v10/index.html'>parameter documentation</a>.
- For a description of the <strong><em>gene sets database files*</em></strong> parameter options, visit <a href="https://www.gsea-msigdb.org/gsea/msigdb/index.jsp">the MSigDB webpage</a>.
- Click <strong><em>Run</em></strong> on the analysis below.</li>
</div>

In [30]:
ssgsea_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00270')
ssgsea_job_spec = ssgsea_task.make_job_spec()
ssgsea_job_spec.set_parameter("input.gct.file", "https://datasets.genepattern.org/data/ccmi_tutorial/2017-12-15/BRCA_HUGO_symbols.preprocessed.gct")
ssgsea_job_spec.set_parameter("output.file.prefix", "")
ssgsea_job_spec.set_parameter("gene.sets.database.files", "ftp://gpftp.broadinstitute.org/module_support_files/msigdb/gmt/c2.cp.wikipathways.v7.2.symbols.gmt")
ssgsea_job_spec.set_parameter("gene.symbol.column", "Name")
ssgsea_job_spec.set_parameter("gene.set.selection", "ALL")
ssgsea_job_spec.set_parameter("sample.normalization.method", "none")
ssgsea_job_spec.set_parameter("weighting.exponent", "0.75")
ssgsea_job_spec.set_parameter("min.gene.set.size", "10")
ssgsea_job_spec.set_parameter("combine.mode", "combine.off")
ssgsea_job_spec.set_parameter("job.memory", "2 Gb")
ssgsea_job_spec.set_parameter("job.queue", "gp-new-beta")
ssgsea_job_spec.set_parameter("job.cpuCount", "1")
ssgsea_job_spec.set_parameter("job.walltime", "02:00:00")
genepattern.display(ssgsea_task)

job62643 = gp.GPJob(genepattern.session.get(0), 62643)
genepattern.display(job62643)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00270')

GPJobWidget(job_number=62643)

## Visualize projected pathways as a heat map

We will use the GenePattern heat map viewer to visualize the resulting projection of genes into pathways.

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
    
- In the **dataset** parameter below, click on the dropdown and select output of the ssGSEA module (it should end with `.PROJ.gct`).
- Click **Run**.
</div>

In [32]:
heatmapviewer_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.visualizer:00010')
heatmapviewer_job_spec = heatmapviewer_task.make_job_spec()
heatmapviewer_job_spec.set_parameter("dataset", "https://beta.genepattern.org/gp/jobResults/62643/BRCA_HUGO_symbols.preprocessed.PROJ.gct")
genepattern.display(heatmapviewer_task)

job62644 = gp.GPJob(genepattern.session.get(0), 62644)
genepattern.display(job62644)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.visualizer:00010')

GPJobWidget(job_number=62644)

## Calculate the ROC for each gene set from the ssGSEA projections and a two-phenotype CLS file

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
    
For the <b>ssGSEA Projection Matrix</b> parameter, click the "Add File or URL..." box and select output of the ssGSEA module (it should end with `.PROJ.gct`).

For the <b>Two phenotype CLS file</b> parameter, click the "Upload File..." button to provide the CLS file with the phenotypic information (<a href="http://software.broadinstitute.org/cancer/software/genepattern/file-formats-guide#CLS">read about CLS files here</a>).<br>Alternatively, drag and drop <a href="https://datasets.genepattern.org/data/module_support_files/DESeq2/BRCA_tumor_and_normal_x40.cls" target="_blank">this sample CLS file</a>.

In [29]:
ssgsea_roc_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:8080.gpserver.ip-172-31-26-71.ip-172-31-26-71.ec2.internal:genepatternmodules:177')
ssgsea_roc_job_spec = ssgsea_roc_task.make_job_spec()
ssgsea_roc_job_spec.set_parameter("PROJ.gct", "https://beta.genepattern.org/gp/jobResults/62643/BRCA_HUGO_symbols.preprocessed.PROJ.gct")
ssgsea_roc_job_spec.set_parameter("CLS", "https://beta.genepattern.org/gp/users/acastanza/tmp/run6706559141305188871.tmp/BRCA_tumor_and_normal_x40.cls")
ssgsea_roc_job_spec.set_parameter("Reverse", "TRUE")
ssgsea_roc_job_spec.set_parameter("Plot.Top.Results", "443")
ssgsea_roc_job_spec.set_parameter("job.memory", "2 Gb")
ssgsea_roc_job_spec.set_parameter("job.queue", "gp-new-beta")
ssgsea_roc_job_spec.set_parameter("job.cpuCount", "1")
ssgsea_roc_job_spec.set_parameter("job.walltime", "02:00:00")
genepattern.display(ssgsea_roc_task)

job62651 = gp.GPJob(genepattern.session.get(0), 62651)
genepattern.display(job62651)

GPTaskWidget(lsid='urn:lsid:8080.gpserver.ip-172-31-26-71.ip-172-31-26-71.ec2.internal:genepatternmodules:177'…

GPJobWidget(job_number=62651)

# References

- Subramanian A, Tamayo P, Mootha VK, Mukherjee S, Ebert BL, Gillette MA, Paulovich A, Pomeroy SL, Golub TR, Lander ES, Mesirov JP. Gene set enrichment analysis: A knowledge-based approach for interpreting genome-wide expression profiles. PNAS. 2005;102(43):15545-15550. http://www.pnas.org/content/102/43/15545.abstract
- Barbie DA, Tamayo P, et al. Systematic RNA interference reveals that oncogenic KRAS-driven cancers require TBK1. Nature. 2009;462:108-112. https://pubmed.ncbi.nlm.nih.gov/19847166/
- MSigDB website (https://www.gsea-msigdb.org/gsea/msigdb/index.jsp)
- GSEA website (https://www.gsea-msigdb.org/gsea/index.jsp)
- Sing T, Sander O, Beerenwinkel N, Lengauer T. ROCR: visualizing classifier performance in R. Bioinformatics. 2005;21(20):3940-3941. https://pubmed.ncbi.nlm.nih.gov/16096348/
